In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/fake_job_postings.csv', engine = 'python')
df = df.drop(['job_id'], axis = 1)
# 대부분 missing 인 col 제거
df = df.drop(['department','salary_range'], axis = 1)
df.head()

,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Marketing Intern,"US, NY, New York","We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland","90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
# location 전처리

print('location null의 fake job posting 비율 : ', df[df['location'].isnull() ==True]['fraudulent'].value_counts()[1]/df[df['location'].isnull() ==True].shape[0])

# 국가명만 남기기
df['location'] = df['location'].apply(lambda x : str(x).split(',')[0])

# mode로 대체
df[df['location'].isnull() ==True]['location'] = df['location'].mode()[0]
# 10개 이상의 관측치 보유 location 만 남기기
loc_val_count = df['location'].value_counts()
loc_over_10 = loc_val_count[loc_val_count>10].index
df = df[df['location'].isin(loc_over_10)].reset_index(drop=True)

print("비율 차이 적으므로 mode로 imputation")


location null의 fake job posting 비율 :  0.05491329479768786
비율 차이 적으므로 mode로 imputation


In [4]:
# required_experience 전처리
print('required_experience 의 unique col 개수(nan포함) :',len(df['required_experience'].unique()))
print('required_experience 의 missing 개수 :',df['required_experience'].isnull().sum())
print(df['required_experience'].value_counts())
# sns.barplot(data=df, x='required_experience',y='fraudulent')

required_experience 의 unique col 개수(nan포함) : 8
required_experience 의 missing 개수 : 6985
required_experience
Mid-Senior level    3772
Entry level         2674
Associate           2278
Not Applicable      1096
Director             385
Internship           380
Executive            140
Name: count, dtype: int64


In [6]:
import datawig

df_train, df_test = datawig.utils.random_split(df)

FileNotFoundError: Could not find module 'C:\Users\glist\PycharmProjects\BigDataAnalysis-HYU-IE\venv\lib\site-packages\mxnet\libmxnet.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [21]:
input_columns = list(set(df.columns) - set(['required_experience']))
target_col_nm = 'required_experience'

imputer = datawig.SimpleImputer(input_columns=input_columns,
                                output_column=target_col_nm,
                                output_path=f'../imputer/{target_col_nm}'
                                )

# imputer.fit_hpo(train_df=df,
#                     learning_rate_candidates=[1e-3, 1e-4],
#                     tokens_candidates=['chars', 'words'],
#                     num_hash_bucket_candidates=[2**10, 2**15],
#                     final_fc_hidden_units=[[100], [150]],
#                     num_epochs=10
#                    )

imputer.fit(train_df=df_train, num_epochs=100,
             batch_size=32)


In [30]:
def impute_train_partially(df, target_col_nm):   
    input_columns = list(set(df.columns) - set([target_col_nm]))

    imputer = datawig.SimpleImputer(input_columns=input_columns,
                                    output_column=target_col_nm,
                                    output_path=f'../imputer/{target_col_nm}'
                                    )
    
    imputer.fit(train_df=df_train, num_epochs=100,
             batch_size=64)

    imputed = imputer.predict(df_test)
    
    return pd.DataFrame(imputed)

In [31]:
import mxnet as mx


def impute_train_whole(df, target_col_nm):   
    input_columns = list(set(df.columns) - set([target_col_nm]))

    imputer = datawig.SimpleImputer(input_columns=input_columns,
                                    output_column=target_col_nm,
                                    output_path=f'../imputer/{target_col_nm}'
                                    )
    
    imputer.fit(train_df=df, num_epochs=100, ctx=mx.gpu(0),
             batch_size=64)

    imputed = imputer.predict(df)
    
    return pd.DataFrame(imputed)

In [41]:
df_train.isna().sum()

location                  0
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        2758
required_experience    5605
required_education     6450
industry               3921
function               5121
fraudulent                0
dtype: int64

In [47]:
impute_result = {}

for target_col_nm in ['employment_type','required_experience','required_education','industry','function']:
    temp = df_train.copy(deep=True)
    imputed = impute_train_whole(temp, target_col_nm)
    na_replaced = temp[target_col_nm].fillna(imputed[f"{target_col_nm}_imputed"], inplace=False)
    impute_result[target_col_nm] = na_replaced   
    